In [1]:
M.shape

In [2]:
from nilearn import image
from __future__ import print_function, division
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from nilearn.plotting import plot_epi
from nilearn import plotting
from nilearn.datasets import load_mni152_template
from nilearn.image import resample_to_img
from nilearn import input_data
from nilearn.connectome import ConnectivityMeasure
import nitime
#Import the time-series objects:
from nitime.timeseries import TimeSeries
#Import the analysis objects:
from nitime.analysis import CorrelationAnalyzer, CoherenceAnalyzer
#Import utility functions:
from nitime.utils import percent_change
from nitime.viz import drawmatrix_channels, drawgraph_channels, plot_xcorr
import pandas as pd

In [3]:
prep_dir="/Volumes/Pegasus_wangyun/ADHD/ITA/MRI/ITA_version1/results/preprocessing"
!find $prep_dir/niftiDATA_*Session*.nii > conn_denoise.csv
F=pd.read_csv("conn_denoise.csv",header=None,names={'files'})
F.shape

In [4]:
M.head()

In [5]:
#matlab matrix in conn is too big to load 
pp='/Volumes/Pegasus_wangyun/ADHD/ITA/MRI/'
M=pd.read_csv(f'{pp}conn_sess_cond.csv')

In [6]:
M.head()

In [7]:
prep_dir="/Volumes/Pegasus_wangyun/ADHD/ADHD_HC_ADD/conn_ADHD_add/results/preprocessing"
!find $prep_dir/niftiDATA_*Session*.nii > hc_conn_denoise.csv
F=pd.read_csv("hc_conn_denoise.csv",header=None,names={'files'})
F.shape

In [8]:
#matlab matrix in conn is too big to load 
#pp='/Volumes/Pegasus_wangyun/ADHD/ITA/MRI/'
M=pd.read_csv('hc_conn_sess_cond.csv')

In [9]:
!ls

In [10]:
!pwd

In [11]:
#matlab matrix in conn is too big to load 
pp='/Users/posnerlab/Documents/GitHub/CBIG/stable_projects/preprocessing/CBIG_fMRI_Preproc2016/HC_prep/'
M=pd.read_csv(f'{pp}hc_conn_sess_cond.csv')

In [12]:
#matlab matrix in conn is too big to load 
pp='/Users/posnerlab/Documents/GitHub/CBIG/stable_projects/preprocessing/CBIG_fMRI_Preproc2016/HC_prep/'
M=pd.read_excel(f'{pp}HC_conn_sess_cond.csv')

In [13]:
#matlab matrix in conn is too big to load 
pp='/Users/posnerlab/Documents/GitHub/CBIG/stable_projects/preprocessing/CBIG_fMRI_Preproc2016/HC_prep/'
M=pd.read_excel(f'{pp}HC_conn_sess_cond.xlsx')

In [14]:
M.head()

In [15]:
##

sub=[]
sess=[]
X1=F[F['files'].str.contains("Session1")].reset_index().drop(columns=['index'])
for i,j in enumerate(X1['files']):
    sub.append('sub'+X1['files'][i].split('/')[-1].split('_')[1][-2:])
    #print(sub)
    sess.append(X1['files'][i].split('/')[-1].split('_')[-1].split('.')[0])
    #print(sess)
X1['ID']=sub

sub=[]
sess=[]
X2=F[F['files'].str.contains("Session2")].reset_index().drop(columns=['index'])
for i,j in enumerate(X2['files']):
    sub.append('sub'+X2['files'][i].split('/')[-1].split('_')[1][-2:])
    #print(sub)
    sess.append(X2['files'][i].split('/')[-1].split('_')[-1].split('.')[0])
    #print(sess)
X2['ID']=sub
X1.merge(X2,how='outer',on='ID')

In [16]:
##

sub=[]
sess=[]
X1=F[F['files'].str.contains("Session1")].reset_index().drop(columns=['index'])
for i,j in enumerate(X1['files']):
    sub.append('sub'+X1['files'][i].split('/')[-1].split('_')[1][-2:])
    #print(sub)
    sess.append(X1['files'][i].split('/')[-1].split('_')[-1].split('.')[0])
    #print(sess)
X1['ID']=sub

sub=[]
sess=[]
X2=F[F['files'].str.contains("Session2")].reset_index().drop(columns=['index'])
for i,j in enumerate(X2['files']):
    sub.append('sub'+X2['files'][i].split('/')[-1].split('_')[1][-2:])
    #print(sub)
    sess.append(X2['files'][i].split('/')[-1].split('_')[-1].split('.')[0])
    #print(sess)
X2['ID']=sub
#X1.merge(X2,how='outer',on='ID')

In [17]:
XX = X1.merge(
    X2, how='outer', on='ID').rename(columns={
        'files_x': 'session1',
        'files_y': 'session2'
    })

In [18]:
DF=M.merge(XX,on='ID')
DF.head()

In [19]:
DD=pd.DataFrame(columns=['ID','pairs'])
keys=('ID','pairs')
for i in np.arange(DF.shape[0]):
    ll=[]
    if i:
        ll.append(DF['session1'][i])
    
    if ll: 
        #print(image.concat_imgs(ll).shape)
        DD=DD.append(pd.Series(dict(zip(keys,(DF['ID'][i],ll)))),ignore_index=True)
DD.rename(columns={'pairs':'post_list'},inplace=True)

In [20]:
DD

In [21]:
DD=pd.DataFrame(columns=['ID','pairs'])
keys=('ID','pairs')
for i in np.arange(DF.shape[0]):
    ll=[]
    if i:
        ll.append(DF['session1'][i])
    
    if ll: 
        #print(image.concat_imgs(ll).shape)
        DD=DD.append(pd.Series(dict(zip(keys,(DF['ID'][i],ll)))),ignore_index=True)
DD.rename(columns={'pairs':'list'},inplace=True)

In [22]:
DD

In [23]:
DD['list'][2]

In [24]:
DD=pd.DataFrame(columns=['ID','pairs'])
keys=('ID','pairs')
for i in np.arange(DF.shape[0]):
    ll=[]
    if DF['session1'][i]:
        ll.append(DF['session1'][i])
    if DF['session2'][i]:
        ll.append(DF['session2'][i])
    
    if ll: 
        #print(image.concat_imgs(ll).shape)
        DD=DD.append(pd.Series(dict(zip(keys,(DF['ID'][i],ll)))),ignore_index=True)
DD.rename(columns={'pairs':'list'},inplace=True)

In [25]:
DD['list'][2]

In [26]:
DD['list'][1]

In [27]:
DF['session2'][i].isnan()

In [28]:
DF['session2'][i].values.isnan()

In [29]:
DF['session2'][i]

In [30]:
import math
print math.isnan(float(DF['session2'][i]))

In [31]:
import math
math.isnan(float(DF['session2'][i]))

In [32]:
DF['session2'][i].isna()

In [33]:
pd.isna(DF['session2'][i])

In [34]:
DD=pd.DataFrame(columns=['ID','pairs'])
keys=('ID','pairs')
for i in np.arange(DF.shape[0]):
    ll=[]
    if ~pd.isna(DF['session1'][i]):
        ll.append(DF['session1'][i])
    if ~pd.isna(DF['session2'][i]):
        ll.append(DF['session2'][i])
    
    if ll: 
        #print(image.concat_imgs(ll).shape)
        DD=DD.append(pd.Series(dict(zip(keys,(DF['ID'][i],ll)))),ignore_index=True)
DD.rename(columns={'pairs':'list'},inplace=True)

In [35]:
DD['list'][1]

In [36]:
All=DDD.merge(DD,on='ID')

In [37]:
DD

In [38]:
DD['list'][1]

In [39]:
~pd.isna(DF['session2'][i])

In [40]:
pd.isna(DF['session2'][i])

In [41]:
!pd.isna(DF['session2'][i])

In [42]:
pd.isna(DF['session2'][i])

In [43]:
not pd.isna(DF['session2'][i])

In [44]:
DD=pd.DataFrame(columns=['ID','pairs'])
keys=('ID','pairs')
for i in np.arange(DF.shape[0]):
    ll=[]
    if not pd.isna(DF['session1'][i]):
        ll.append(DF['session1'][i])
    if not pd.isna(DF['session2'][i]):
        ll.append(DF['session2'][i])
    
    if ll: 
        #print(image.concat_imgs(ll).shape)
        DD=DD.append(pd.Series(dict(zip(keys,(DF['ID'][i],ll)))),ignore_index=True)
DD.rename(columns={'pairs':'list'},inplace=True)

In [45]:
DD['list'][1]

In [46]:
FC=[]
Cohe=[]

for i in np.arange(DD.shape[0]):
    
    # concatenate images for post and pre separately
     #ll=[DF['session1'][i],DF['session2'][i]]
    #X1 = image.load_img(DF['session1'][i]) # post session1
    #X2 = image.load_img(DF['session2'][i]) # post session2
   
    spheres_masker = input_data.NiftiSpheresMasker(
        seeds=coords, smoothing_fwhm=4, radius=5.,detrend=True,
     low_pass=0.1, high_pass=0.01, t_r=2.2,memory='nilearn_cache', memory_level=1)
    X=image.concat_imgs(DD['list'][i])

    print(X.shape)
    time_series2 = spheres_masker.fit_transform(X)
    correlation_measure = ConnectivityMeasure(kind='correlation')
    correlation_matrix = correlation_measure.fit_transform([time_series2])[0]
    FC.append(correlation_matrix)
    data = percent_change(time_series2.T)
    T = TimeSeries(data, sampling_interval=DF['TR'][i], time_unit='s')
    C =CoherenceAnalyzer(T)
    f_lb=0.01
    f_ub=0.1
    freq_idx = np.where((C.frequencies > f_lb) * (C.frequencies < f_ub))[0]

    Cohe.append(np.mean(C.coherence[:, :, freq_idx], -1))  # Averaging on the last dimension
    #fig03 = drawmatrix_channels(coh,  size=[10., 10.], color_anchor=0)
np.save('power_correlation_HC.npy', FC)
np.save('power_coherence_HC.npy', Cohe)

In [47]:
from nilearn import datasets
power = datasets.fetch_coords_power_2011()
print('Power atlas comes with {0}.'.format(power.keys()))

import numpy as np

coords = np.vstack((power.rois['x'], power.rois['y'], power.rois['z'])).T

print('Stacked power coordinates in array of shape {0}.'.format(coords.shape))

from nilearn import input_data

spheres_masker = input_data.NiftiSpheresMasker(
    seeds=coords, smoothing_fwhm=4, radius=5.,detrend=True,
 low_pass=0.1, high_pass=0.01, t_r=2.2,memory='nilearn_cache', memory_level=1)
msdl_data = datasets.fetch_atlas_msdl()
msdl_coords = msdl_data.region_coords
n_regions = len(msdl_coords)
masker = input_data.NiftiMapsMasker(msdl_data.maps, resampling_target="data", t_r=2.2, detrend=True,
    low_pass=.1, high_pass=.01, memory='nilearn_cache', memory_level=1)

In [48]:
FC=[]
Cohe=[]

for i in np.arange(DD.shape[0]):
    
    # concatenate images for post and pre separately
     #ll=[DF['session1'][i],DF['session2'][i]]
    #X1 = image.load_img(DF['session1'][i]) # post session1
    #X2 = image.load_img(DF['session2'][i]) # post session2
   
    spheres_masker = input_data.NiftiSpheresMasker(
        seeds=coords, smoothing_fwhm=4, radius=5.,detrend=True,
     low_pass=0.1, high_pass=0.01, t_r=2.2,memory='nilearn_cache', memory_level=1)
    X=image.concat_imgs(DD['list'][i])

    print(X.shape)
    time_series2 = spheres_masker.fit_transform(X)
    correlation_measure = ConnectivityMeasure(kind='correlation')
    correlation_matrix = correlation_measure.fit_transform([time_series2])[0]
    FC.append(correlation_matrix)
    data = percent_change(time_series2.T)
    T = TimeSeries(data, sampling_interval=DF['TR'][i], time_unit='s')
    C =CoherenceAnalyzer(T)
    f_lb=0.01
    f_ub=0.1
    freq_idx = np.where((C.frequencies > f_lb) * (C.frequencies < f_ub))[0]

    Cohe.append(np.mean(C.coherence[:, :, freq_idx], -1))  # Averaging on the last dimension
    #fig03 = drawmatrix_channels(coh,  size=[10., 10.], color_anchor=0)
np.save('power_correlation_HC.npy', FC)
np.save('power_coherence_HC.npy', Cohe)

In [49]:
plt.hist(FC[31].ravel(),bins=200)
plt.hist(Cohe[31].ravel(),alpha=0.6,bins=200)
plt.show()

In [50]:
plt.hist(FC[31].ravel(),bins=200)
plt.hist(Cohe[31].ravel(),alpha=0.6,bins=200)
plt.show()

In [51]:
plt.hist(FC[1].ravel(),bins=200)
plt.hist(Cohe[1].ravel(),alpha=0.6,bins=200)
plt.show()

In [52]:
<style>
td {
  font-size: 50px
}
</style>

In [53]:
import os
from subprocess import check_call

def post_save(model, os_path, contents_manager):
    """post-save hook for converting notebooks to .py scripts"""
    if model['type'] != 'notebook':
        return # only do this for notebooks
    d, fname = os.path.split(os_path)
    check_call(['ipython', 'nbconvert', '--to', 'script', fname], cwd=d)

In [54]:
post_save('notebook','/Users/posnerlab/Documents/GitHub/ITA')

In [55]:
post_save('notebook','/Users/posnerlab/Documents/GitHub/ITA',contents_manager)

In [56]:
import io
import os
from notebook.utils import to_api_path

_script_exporter = None

def script_post_save(model, os_path, contents_manager, **kwargs):
    """convert notebooks to Python script after save with nbconvert

    replaces `jupyter notebook --script`
    """
    from nbconvert.exporters.script import ScriptExporter

    if model['type'] != 'notebook':
        return

    global _script_exporter

    if _script_exporter is None:
        _script_exporter = ScriptExporter(parent=contents_manager)

    log = contents_manager.log

    base, ext = os.path.splitext(os_path)
    script, resources = _script_exporter.from_filename(os_path)
    script_fname = base + resources.get('output_extension', '.txt')
    log.info("Saving script /%s", to_api_path(script_fname, contents_manager.root_dir))

    with io.open(script_fname, 'w', encoding='utf-8') as f:
        f.write(script)

c.FileContentsManager.post_save_hook = script_post_save

In [57]:
%notebook "/Users/posnerlab/Documents/GitHub/ITA/filename.ipynb"     

In [58]:
%notebook /Users/posnerlab/Documents/GitHub/ITA/filename.ipynb     

In [59]:
%notebook /Users/posnerlab/Documents/GitHub/ITA/Poweratlas_Coherence_extract.ipynb     